# shp2html

This script converts a shapefile with polygons to an HTML image map.

Input requirement:



### How to prepare Python environment

Requirement for python environment

1. Install native GDAL
    On Mac or unix-like system

    `sudo apt-get install gdal-bin`

    On macOS with homebrew

    `brew install gdal`

2. Create Python virtual environment 

    `virtualenv metpetools` 

3. Activate virtual environment

    `source ./bin/activate`

4. Install python dependencies

    `pip install GDAL`
    
    `pip install jupyter`


## RUN

Put your raster and your shapefile in a subfolder in this python environment

Then change in the nex cell the User inputs strings. Change only path and filename in
SHAPEFILE, RASTER and assign a NAME to the project, this name will be used for output zip file.

In [1]:
import json
import os

from zipfile import ZipFile
from osgeo import ogr


# !!! User inputs !!!
# Change this paths with the path to your shapefile and raster
SHAPEFILE = r'sample_data/TOR1/TOR1.shp'
RASTER = r'sample_data/TOR1/TOR1.jpg'
NAME = 'TEST' # name of the project, used to name the output files
# !!! End of user inputs !!!

Now run the code below

In [ ]:
def fix_coords(coords, origin='top-left'):
    '''
    Transform floating coordinates to integers and fixes their values based on
    origin. Removes the last couple if it coincides with the first.

    Parameters
    ----------
    coords : list
        List of (x, y) tuples.
    origin : str, optional
        Origin of the coordinates. Only the 'top-left' origin is currently
        available. The default is 'top-left'.

    Raises
    ------
    NotImplementedError
        Only 'top-left' origin is implemented.

    Returns
    -------
    fixed : list
        List of fixed (x, y) tuples.

    '''
    # Remove duplicated couples
    if coords[0] == coords[-1]:
        del coords[-1] 

    if origin == 'top-left':
        fixed = [(abs(round(x)), abs(round(y))) for x, y in coords]
        return fixed
    else:
        raise NotImplementedError(f"Can't convert coords with origin {origin}")


# Set a global variable that tracks the occurrence of inner rings in polygons
INRINGS = 0

# Set a global variable that holds the list of vertices of each polygon
polygons_vert = []
# set a global variable that holds the list of attributes of each polygon
attributes_list = []

# Open the shapefile
shapefile = ogr.Open(SHAPEFILE)

# Get the layer
layer = shapefile.GetLayer()

# Iterate over the features in the layer
for feature in layer:
    attributes = feature.items() # get attributes
    attributes_list.append(attributes) # append attributes to list
    geometry = feature.geometry()

    # Track inner rings
    n_rings = geometry.GetGeometryCount()
    if n_rings > 1:
        INRINGS += n_rings-1

    # We only get vertices of outer ring (= first element of GetGeometryRef)
    vertices = geometry.GetGeometryRef(0).GetPoints()
    polygons_vert.append(fix_coords(vertices))


# Send warning if inner rings (holes) have been detected
if INRINGS:
    print(f'Warning: {INRINGS} holes detected. Not handled.')

# HTML header and footer part
HTMLHEAD = '<!DOCTYPE html>\n<html>\n<head>\n\t<title>Image Map</title>\n'\
    '\t<meta charset="utf-8">\n\t<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>\n'\
    '\t<script src="https://cdnjs.cloudflare.com/ajax/libs/image-map-resizer/1.0.10/js/imageMapResizer.min.js"></script>\n'\
        '</head>\n<body>\n'   
HTMLFOOT = '<script>\n\t$(document).ready(function() {\n\t $(\'map\').imageMapResize();\n\t}); \n</script>\n</body>\n</html>'    

    
image_block = f'<img src="{RASTER}" usemap="#image-map" '\
               'style="width: 100%; height: auto;">\n'

HTMLPOLYGONS = '<map name="image-map">\n'
for a, p in zip(attributes_list, polygons_vert):
    coordstr = str(p)[1:-1].replace('(','').replace(')','').replace(' ','')
    id_ = a['OBJECTID']
    line = f'\t<area target="_blank" id="imgzone{id_}" '\
           f'alt="Element #{id_}" title="Element #{id_}" '\
           f'coords="{coordstr}" shape="poly">\n'
    HTMLPOLYGONS += line
HTMLPOLYGONS += '</map>\n'

html_path = rf'{NAME}.html'
AttributeTable_path = rf'{NAME}.json'

# Save as file
with open(html_path, 'w', encoding="utf-8") as out_file:
    out_file.write(HTMLHEAD)
    out_file.write(image_block)
    out_file.write('\n')
    out_file.write(HTMLPOLYGONS)
    out_file.write(HTMLFOOT)

with open(f'{NAME}.json', 'w', encoding="utf-8") as f:
    json.dump(attributes_list, f)


# Create zip file
with ZipFile(rf'{NAME}.zip', 'w') as outzip:
    outzip.write(RASTER)
    outzip.write(html_path)
    outzip.write(AttributeTable_path)

# Remove files
os.remove(html_path)
os.remove(AttributeTable_path)